<a href="https://colab.research.google.com/github/quantumhome/DataAnalysisCaseStudy/blob/master/Silver_Assignment_4_Tool_Chain_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 4  Create a toolchain with LLM
langchain and nosql database such as mongodb or redis and upload your information on the nosql database and query it through LLM**

In [1]:
!pip install -U langchain langchain-community langchain-huggingface langchain-redis langchain-classic
!service redis-server stop > /dev/null
!apt-get purge redis-server -y > /dev/null
!curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
!echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis-stack.list
!sudo apt-get update > /dev/null
!sudo apt-get install redis-stack-server -y > /dev/null
!redis-stack-server --daemonize yes
!pip install redis > /dev/null

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [2]:
import os
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain_redis import RedisVectorStore
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

# Fix: Import from langchain_classic for legacy chain support
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from google.colab import userdata
# 1. API Configuration
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
REDIS_URL = "redis://localhost:6379"


# 2. Setup Models
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use ChatHuggingFace wrapper to handle the ChatPromptTemplate correctly
llm_engine = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta", # Changed to a currently supported model
    task="text-generation",
    max_new_tokens=512,
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)
llm = ChatHuggingFace(llm=llm_engine)

# 3. Upload Information to Redis
docs = [
    Document(page_content="The user has successfully bypassed Docker by installing Redis natively.", metadata={"id": 1}),
    Document(page_content="Redis is currently acting as a NoSQL vector database at localhost:6379.", metadata={"id": 2}),
    Document(page_content="LangChain 2025 uses create_retrieval_chain for modern RAG pipelines.", metadata={"id": 3})
]

vector_store = RedisVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    redis_url=REDIS_URL,
    index_name="my_native_index"
)

# 4. Define the Retrieval Logic
# Increased k to get more documents and debug retrieval issues
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

system_prompt = (
    "You are a helpful assistant. Use the following context to answer the question. "
    "If the answer isn't in the context, say you don't know.\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# 5. Assemble and Run the Chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# Run the query
response = rag_chain.invoke({"input": "in which port redis is running?"})
print(f"--- LLM Response ---\n{response['answer']}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- LLM Response ---


To find the port number of Redis running on your local machine, you can use the netstat command.

```
$ sudo netstat -plnt | grep redis
```

Output:

```
tcp        0      0 127.0.0.0.1:6379               *:*                   LISTEN      13997/redis
```

The above output indicates that Redis is running on port number 6379.

So, Redis is running on localhost:6379.

To confirm, you can also run `redis-cli --version` command to see the mentioned port under `redis-cli`.

![image](https://user-images.githubusercontent.com/787771836/155744506116617-47050c58e55f3c0d89529e3f87c5d2485fdd2b664697e3e64588a1e245c5e8693bf3e6c.png)

That means Redis is currently acting as a NoSQL vector database on the port 6379.

LangChain uses redis as a NoSQL database to store embeddings and metadata for vector search.

LangChain 2025 uses `create_retrieval_chain` to perform vector search. `create_retrieval_chain` is a helper function used to create a `Retriever`.

`create_retrieval_chain`